<a href="https://colab.research.google.com/github/rato42/linear_regression/blob/unstable-yeah-sure/CLASSBASED_new_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [218]:
import sys

# Detect Colab environment
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    from google.colab import drive, auth
    from googleapiclient.discovery import build
    from googleapiclient.http import MediaIoBaseDownload

    auth.authenticate_user()
    drive.mount("/content/drive")
else:
    from oauth2client.service_account import ServiceAccountCredentials

    print("Running outside Colab (VS Code/local)")

try:
    import gspread
except ModuleNotFoundError:
    if IN_COLAB and "google.colab" in str(get_ipython()):
        %pip install gspread
    import gspread

from gspread_dataframe import set_with_dataframe

Running outside Colab (VS Code/local)


In [219]:

import os

import time

import math
import scipy.stats as stats
import pandas as pd
import numpy as np

import pwlf

from scipy.stats import linregress, ttest_ind, ttest_rel, ttest_1samp
import matplotlib.pyplot as plt
import re
from matplotlib.font_manager import FontProperties
import statsmodels.formula.api as smf
import statsmodels.api as sm

from datetime import datetime, timedelta

from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tools.tools import add_constant
from statsmodels.tsa.arima.model import ARIMA

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer

from dateutil.relativedelta import relativedelta

from sklearn import set_config
set_config(display="diagram")

import logging
from typing import Dict, Any

import io

# LOAD --- data from gsheet if colab or local


In [220]:
dados_brutos = None
output_wksheet = None

input_path = "https://docs.google.com/spreadsheets/d/1m3eEaxWT4Unb8jBZWKjiLivavfA0x3PT1F1Rz1eXwVE/edit?gid=0"
output_path = "https://docs.google.com/spreadsheets/d/1RzC3DfKNUwYA-qfUN0i53hUwRkWGMBOCoYQqdqG4MQY/edit?gid=0"

local_input_path = (
    "D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\input\\input.csv" ### poa
    #"D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\input\\input.xlsx" ### rs
)

if IN_COLAB:
    import gspread
    from google.auth import default

    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    workbook = gc.open_by_url(input_path)
    worksheets = workbook.worksheets()
    output_wksheet = gc.open_by_url(output_path)
    dados_brutos = worksheets[0].get_all_values()
else:
    try:
        dados_brutos = pd.read_csv(local_input_path, header=None).values.tolist()
    except:
        dados_brutos = pd.read_excel(local_input_path, header=None).values.tolist()

# ARGS --- Define Args


fit_kwargs = {
    "cov_type": "HAC",
    "cov_kwds": {"maxlags": 6}  # Account for ~6 month lagged effects
}

✓ Autocorrelation from policy inertia
✓ Seasonal patterns in mental health data
✓ Lagged treatment effects

In [221]:
INDEPENDENT_VAR = "periodo"  ### Y
DEPENDENT_VAR = "valores"  ### X
DATE_NAME = "date"
VARIABLE_NAME = "variavel"
SAVE_ARG = True

PANDEMIA_START_DATE = "2020-01-01"
PANDEMIA_END_DATE = "2020-09-01"
POS_PANDEMIA_START_DATE = "2020-12-01"#"2021-02-01"#




LOCAL_OUTPUT_PATH = "D:\\CodeStuff\\Stats\\colab_linear_regression\\linear_regression\\output\\output.xlsx"

DUMMY_ARGS =        {
        #'PrePandemia': {"start_date": "2016-01-01", "end_date": "2019-12-31"},
        #"Pandemia": {"start_date": "2020-01-01", "end_date": "2020-12-31"},
        #"PosPandemia": {"start_date": "2021-01-01", "end_date": "2022-06-30"},
        'Pulse_Pandemia': {"start_date": PANDEMIA_START_DATE, "end_date": PANDEMIA_END_DATE},
        #'Step_PandemiaePos': {"start_date": PANDEMIA_START_DATE, "end_date": "2022-06-30"},
        'Trend_PandemiaePos' : {"start_date": POS_PANDEMIA_START_DATE, "end_date": "2024-12-31", 'add_time_trend': True}, 
        }


MODEL_FORMULA = DEPENDENT_VAR + "~" + INDEPENDENT_VAR
if DUMMY_ARGS:
    for key in DUMMY_ARGS.keys():
        MODEL_FORMULA += " + " + key

ARGS = {
    # data preprocessing
    #"PeriodosInclusivos_Range": [2015, 2022],  # @[Y1, Y2, M1, M2, D1,D2] start and stop
    #"PeriodGrouping": 3,
    ################### TODO: Need to make a check for multiples of periodgrouping
    #"FrequencyDateGroup": "QE",
    'FrequencyDateGroup' : 'QE',#'Q',
    #"ScaleDataValues": True,
    #"DatesAsOrdinal": False,
    #"MeltDF": True,

    # regression
    "RegressionMethod": "prais_winsten",  #'ols' or 'prais_winsten' or 'arima'
    #"PeriodosExcluirDaRegressao": [2020, 2021],
    "ModelFormula": MODEL_FORMULA,
    "IterativeFit": True,
    "MaxIterations": 100,
    "CriterioBestRho": "aic",  #'aic' or 'bic'
    'best_rho_range': [-1.0, 1.0, 0.01],  # range of rho to test
    ### checar se deveria usar essa covariancia, n creio q tenha heteorsolskdkaikdoaktaiticia nos meus dados
    'CovType': 'HAC',
    'CovKwargs': {'maxlags': 6},
    'ARIMAOrder': (1, 0, 0),  # ARIMA order (p, d, q)

    # prediction
    "additional_dates_factor": 0.0,

    'DfsForRegression' : {
        #"Pandemia": [2016, 2020],
        #"PrePandemia": [2016, 2019],
        "Total": [2016, 2024],
    },
    'DummyArgs' : DUMMY_ARGS,
        # Add more as needed (e.g., 'PrePandemia': {'start_date': '2016-01-01', 'end_date': '2019-12-31'})

    'DoChowTest' : False,
    # debug
    #'VerboseResultsAnalysis': True,      ### needs reimplementation
}



# Define Preprocessors


In [222]:
def BuildDataDF(data):
    string_dates = data[0][1:]  # Store original string dates

    # Converter as strings de datas em objetos datetime
    #date_list = [datetime.strptime(date, "%d/%m/%Y") for date in string_dates]
    date_list = [
    date if isinstance(date, datetime) else datetime.strptime(date, "%d/%m/%Y")
    for date in string_dates
    ]

    # Extrair os nomes das variáveis
    variavel = [row[0] for row in data[1:]]

    # Extrair os dados (sem agrupamento)
    dados = [row[1:] for row in data[1:]]
    dados = np.array(dados, dtype=int)  # Converter para tipo object para evitar warning

    # Criar o DataFrame with DatetimeIndex
    df = pd.DataFrame(data=dados, index=variavel, columns=pd.DatetimeIndex(date_list))

    return df

class DateFilterTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self, periodos_inclusivos=None, periodos_exclusivos=None, date_col=DATE_NAME
    ):
        self.periodos_inclusivos = periodos_inclusivos
        self.periodos_exclusivos = periodos_exclusivos
        self.date_col = date_col

    def get_date_range(self, Y1, Y2=None, M1=None, M2=None, D1=None, D2=None):
        Y2 = Y2 or Y1
        M1 = M1 or 1
        M2 = M2 or 12
        D1 = D1 or 1
        D2 = D2 or 31
        start_date = datetime(Y1, M1, D1)
        end_date = datetime(Y2, M2, D2)
        return pd.date_range(start=start_date, end=end_date, freq="D")

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()

        # Convert date column if it's not datetime yet
        if not pd.api.types.is_datetime64_any_dtype(X[self.date_col]):
            X[self.date_col] = pd.to_datetime(X[self.date_col], errors="coerce")

        if self.periodos_inclusivos:
            valid_dates = self.get_date_range(*self.periodos_inclusivos)
            return X[X[self.date_col].isin(valid_dates)].reset_index(drop=True)

        elif self.periodos_exclusivos:
            invalid_dates = self.get_date_range(*self.periodos_exclusivos)
            return X[~X[self.date_col].isin(invalid_dates)].reset_index(drop=True)

        return X

class PeriodGroupingTransformer(BaseEstimator, TransformerMixin):
    """
    Transformer for grouping data by periods (e.g., months, quarters).

    This transformer takes a DataFrame with datetime columns and groups
    the data by specified periods, summing the values within each group.

    ARGS:
        period_grouping (int): The number of periods to group together.
                               For example, 3 for quarterly grouping.
                               Defaults to 2.
    """

    def __init__(self, period_grouping=2):
        self.period_grouping = period_grouping

    def fit(self, X, y=None):
        """
        This transformer does not require fitting.
        """
        return self

    def transform(self, X):
        """
        Groups the data by specified periods and sums the values.

        ARGS:
            X (pd.DataFrame): The input DataFrame with datetime columns.

        Returns:
            pd.DataFrame: The grouped DataFrame with summed values.
                          Columns are datetime objects representing the end of each group.
        """
        if self.period_grouping == 1:
            return X
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame.")

        # Convert columns to datetime if they are not already
        #if not isinstance(X.columns[0], pd.Timestamp):
        #    X.columns = pd.to_datetime(X.columns, errors="coerce")

        grouped_data = {}
        
        # Iterate over columns, grouping by period_grouping
        for i in range(0, len(X.columns), self.period_grouping):
            group_cols = X.columns[
                i : i + self.period_grouping
            ]  # Select columns for the current group
            group_name = group_cols[
                0
            ]  # group_cols[-1] Use the last column's datetime as the group name
            grouped_data[group_name] = X[group_cols].sum(
                axis=1
            )  # Sum values within the group

        grouped_df = pd.DataFrame(grouped_data)

        # Ensure columns remain as datetime objects
        grouped_df.columns = pd.to_datetime(grouped_df.columns)
        return grouped_df

class MeltTransformer(BaseEstimator, TransformerMixin):
    def __init__(
        self,
        var_name=VARIABLE_NAME,
        value_name=DEPENDENT_VAR,
        date_name=DATE_NAME,
        y_name=INDEPENDENT_VAR,
    ):
        self.var_name = var_name
        self.value_name = value_name
        self.date_name = date_name
        self.y_name = y_name

    def fit(self, X, y=None):
        # Nothing to learn, so just return self
        return self

    def transform(self, X):
        # Transpose and reset index
        df_long = X.T.reset_index()

        # Rename index to date_name
        df_long = df_long.rename(columns={"index": self.date_name})

        # Melt the DataFrame
        df_long = pd.melt(
            df_long,
            id_vars=[self.date_name],
            var_name=self.var_name,
            value_name=self.value_name,
        )

        # Sort by category and date
        df_final = df_long.sort_values([self.var_name, self.date_name]).reset_index(
            drop=True
        )

        return df_final

class PeriodIndexAdder(BaseEstimator, TransformerMixin):
    def __init__(self, group_col=VARIABLE_NAME, new_col=INDEPENDENT_VAR):
        self.group_col = group_col
        self.new_col = new_col
        self.period_date_mapping = {}  # Dictionary to store period-date equivalences

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.new_col] = X.groupby(self.group_col).cumcount() + 1

        # Store the mapping of periods to dates
        for group, group_data in X.groupby(self.group_col):
            self.period_date_mapping[group] = dict(zip(group_data[DATE_NAME], group_data[self.new_col]))

        return X

class DummyVariableAdder(BaseEstimator, TransformerMixin):
    """
    Adds dummy variables to a DataFrame based on a date threshold or condition.

    Parameters:
    -----------
    date_column : str
        Name of the column containing dates (if using date-based dummies).
    start_date : str (optional)
        Start date for the dummy period (e.g., '2020-03-01').
    end_date : str (optional)
        End date for the dummy period (e.g., '2022-12-31').
    dummy_name : str
        Name of the new dummy column (default: 'is_pandemic').
    condition : callable (optional)
        Custom function to define the dummy condition (e.g., lambda x: x > 100).
    """

    def __init__(
        self,
        date_column=None,
        start_date=None,
        end_date=None,
        dummy_name="is_pandemic",
        condition=None,
        add_time_trend=False,
    ):
        self.date_column = date_column
        self.start_date = start_date
        self.end_date = end_date
        self.dummy_name = dummy_name
        self.condition = condition
        self.add_time_trend = add_time_trend

    def fit(self, X, y=None):
        # No fitting needed for this transformer
        return self

    def transform(self, X):
        X = X.copy()

        # Create dummy based on dates
        if self.date_column is not None and self.start_date is not None:
            dates = pd.to_datetime(X[self.date_column])
            if self.end_date is not None:
                X[self.dummy_name] = (
                    (dates >= pd.to_datetime(self.start_date))
                    & (dates <= pd.to_datetime(self.end_date))
                ).astype(int)
            else:
                X[self.dummy_name] = (dates >= pd.to_datetime(self.start_date)).astype(
                    int
                )

        # Create dummy based on a custom condition
        elif self.condition is not None:
            X[self.dummy_name] = X.apply(self.condition, axis=1).astype(int)

        else:
            raise ValueError(
                "Either `date_column` + `start_date` or `condition` must be provided."
            )

        return X

class DataFrameSorter(BaseEstimator, TransformerMixin):
    """
    Sorts a DataFrame by a specified column.

    Parameters:
    -----------
    sort_by : str
        Name of the column to sort by.
    ascending : bool, default=True
        Whether to sort in ascending order.
    """

    def __init__(self, sort_by, ascending=True):
        self.sort_by = sort_by
        self.ascending = ascending

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        if self.sort_by not in X.columns:
            raise ValueError(f"Column '{self.sort_by}' not found in DataFrame.")
        return X.sort_values(by=self.sort_by, ascending=self.ascending).reset_index(drop=True)

In [223]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd
import numpy as np

class MultiDummyAdder(BaseEstimator, TransformerMixin):
    """
    Adds multiple dummy variables based on date ranges or custom conditions.
    Allows each dummy to be binary or a local time trend via its own config.

    Parameters:
    -----------
    dummy_config : dict
        Dictionary defining dummy variables. Example:
        {
            'Pandemia': {
                'start_date': '2020-01-01',
                'end_date': '2020-12-31',
                'add_time_trend': True
            }
        }
    date_column : str
        Name of the column containing dates (must be datetime).
    """

    def __init__(self, dummy_config, date_column="date"):
        self.dummy_config = dummy_config
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.copy()
        X[self.date_column] = pd.to_datetime(X[self.date_column])
        X = X.sort_values(self.date_column).reset_index(drop=True)

        for dummy_name, config in self.dummy_config.items():
            start_date = pd.to_datetime(config.get("start_date"))
            end_date = pd.to_datetime(config.get("end_date"))
            use_trend = config.get("add_time_trend", False)

            if not start_date:
                raise ValueError(f"Missing 'start_date' for dummy '{dummy_name}'.")

            if end_date is not None:
                mask = (X[self.date_column] >= start_date) & (X[self.date_column] <= end_date)
            else:
                mask = X[self.date_column] >= start_date

            if use_trend:
                start_periodo = X.loc[mask, INDEPENDENT_VAR].min()
                X[dummy_name] = 0
                X.loc[mask, dummy_name] = (X.loc[mask, INDEPENDENT_VAR] - start_periodo + 1).clip(lower=0)
            else:
                X[dummy_name] = mask.astype(int)

        return X


In [224]:
class FrequencyGroupingTransformer(BaseEstimator, TransformerMixin):
    """
    ## only datetime supported groupings
    Transformer for grouping data by standard datetime frequencies.

    This transformer takes a DataFrame with datetime columns and groups
    the data by a specified frequency (e.g., 'Q' for quarterly, 'M' for monthly).

    ARGS:
        frequency (str): The frequency to group by. Defaults to 'Q' (quarterly).
                        Supported frequencies are those accepted by pd.Grouper (e.g., 'Q', 'M', 'Y').
    """

    def __init__(self, frequency="Q"):  # Default to quarterly grouping
        self.frequency = frequency

    def fit(self, X, y=None):
        """
        This transformer does not require fitting.

        ARGS:
            X (pd.DataFrame): The input DataFrame.
            y (Any, optional): Ignored. Defaults to None.

        Returns:
            self: Returns the transformer instance.
        """
        return self

    def transform(self, X):
        """
        Groups the data by the specified frequency and sums the values.

        ARGS:
            X (pd.DataFrame): The input DataFrame with datetime columns.

        Returns:
            pd.DataFrame: The grouped DataFrame with summed values.
                          Columns are datetime objects representing the end of each group.
        """
        if self.frequency == None or self.frequency == 'M':
            return X
        if not isinstance(X, pd.DataFrame):
            raise TypeError("Input must be a pandas DataFrame.")



        # Convert columns to datetime if they are not already
        if not isinstance(X.columns[0], pd.Timestamp):
            X.columns = pd.to_datetime(X.columns, errors="coerce")

        # Group by the specified frequency using pd.Grouper
        grouped_df = X.T.groupby(pd.Grouper(freq=self.frequency)).sum().T

        return grouped_df



In [225]:

def PopulateRegressionDfsTable(raw_data_df: pd.DataFrame, dfs_for_regression: Dict):
    regression_dfs_table = {}   
    period_date_map = None
    for key, date_range in dfs_for_regression.items():
        # Infer the period name (e.g., remove 'DateRange' suffix)

        pipe = Pipeline(
            [
        # ('scaler', StandardScaler()), ### excluir dpois?
        ("grouping",
            FrequencyGroupingTransformer(frequency=ARGS['FrequencyDateGroup']),
        ),
        ("melt", MeltTransformer()),
        
        (
            "date_filter",
            DateFilterTransformer(periodos_inclusivos=date_range),
        ),

        #(
        #    "period_grouping",
        #    PeriodGroupingTransformer(period_grouping=ARGS["PeriodGrouping"]),
        #),
        #(

        ("period_index_adder", PeriodIndexAdder()),
        ("dummy_adder", MultiDummyAdder(dummy_config=ARGS['DummyArgs'])),

        ("sorter", DataFrameSorter(sort_by=VARIABLE_NAME, ascending=True)),
            ]
        )
        regression_dfs_table[key] = pipe.fit_transform(raw_data_df)
        if period_date_map is None:
            for key, value in pipe.named_steps['period_index_adder'].period_date_mapping.items():
                period_date_map = value
                break
        

    return regression_dfs_table, period_date_map

# **EXEC** --- Build DF and Apply Transformers






In [226]:
RawDataDF = BuildDataDF(dados_brutos)
RegressionDfsTable, PERIOD_DATE_MAP = PopulateRegressionDfsTable(RawDataDF, ARGS['DfsForRegression'])

# **Get Period Indexes**

In [227]:
def find_idx_from_date(target_date, period_date_map=PERIOD_DATE_MAP):
    """
    Finds the 'periodo' (index) corresponding to a given datetime in the PERIOD_DATE_MAP.

    Args:
        target_date (str or datetime): The target date to search for (e.g., '2020-01-01').
        period_date_map (dict): The PERIOD_DATE_MAP containing period-to-date mappings.

    Returns:
        int: The 'periodo' (index) corresponding to the target date, or None if not found.
    """
    # Ensure the target_date is a datetime object and normalize to quarter-end
    if isinstance(target_date, str):
        target_date = pd.to_datetime(target_date)
    if ARGS['FrequencyDateGroup'] == 'Q' or ARGS['FrequencyDateGroup'] == 'QE':
        target_date = target_date + pd.offsets.QuarterEnd(0)

    # Iterate through the PERIOD_DATE_MAP to find the matching period
    for date, period in period_date_map.items():
        if isinstance(date, str):
            date = pd.to_datetime(date)
        if ARGS['FrequencyDateGroup'] == 'Q' or ARGS['FrequencyDateGroup'] == 'QE':
            date = date + pd.offsets.QuarterEnd(0) # Normalize to quarter-end
        if date == target_date:
            return period - 1

    return None


def find_date_from_idx(target_idx, period_date_map=PERIOD_DATE_MAP):
    """
    Finds the date corresponding to a given 'periodo' (index) in the PERIOD_DATE_MAP.

    Args:
        target_idx (int): The target index to search for (e.g., 1, 2, 3).
        period_date_map (dict): The PERIOD_DATE_MAP containing period-to-date mappings.

    Returns:
        datetime or None: The date corresponding to the target index, or None if not found.
    """
    # Iterate through the PERIOD_DATE_MAP to find the matching index
    for date, period in period_date_map.items():
        if period == target_idx:
            date = pd.to_datetime(date) if isinstance(date, str) else date
            date = date + pd.offsets.QuarterEnd(0) if ARGS['FrequencyDateGroup'] == 'Q' or ARGS['FrequencyDateGroup'] == 'QE' else date
            return date
    return None

In [228]:
PANDEMIC_START_PERIOD_INDEX = find_idx_from_date(PANDEMIA_START_DATE, PERIOD_DATE_MAP)
PANDEMIC_END_PERIOD_INDEX = find_idx_from_date(PANDEMIA_END_DATE, PERIOD_DATE_MAP)
POS_PANDEMIC_START_PERIOD_INDEX = find_idx_from_date(POS_PANDEMIA_START_DATE, PERIOD_DATE_MAP)
PANDEMIC_START_PERIOD_INDEX,PANDEMIC_END_PERIOD_INDEX, POS_PANDEMIC_START_PERIOD_INDEX


(16, 18, 19)

# EXPLORATION --- Chow Test


In [229]:
main_regrdf = RegressionDfsTable['Total']
main_regrdf[VARIABLE_NAME]
main_regrdf

,date,variavel,valores,periodo,Pulse_Pandemia,Trend_PandemiaePos
0,2016-03-31,"(F00 - F09) Transtornos mentais orgânicos, inc...",431,1,0,0
1,2016-06-30,"(F00 - F09) Transtornos mentais orgânicos, inc...",516,2,0,0
2,2021-03-31,"(F00 - F09) Transtornos mentais orgânicos, inc...",755,21,0,2
3,2020-12-31,"(F00 - F09) Transtornos mentais orgânicos, inc...",788,20,0,1
4,2016-09-30,"(F00 - F09) Transtornos mentais orgânicos, inc...",663,3,0,0
...,...,...,...,...,...,...
645,2020-09-30,zzGrand Total,19650,19,0,0
646,2018-03-31,zzGrand Total,18576,9,0,0
647,2017-09-30,zzGrand Total,20283,7,0,0
648,2016-03-31,zzGrand Total,14800,1,0,0


In [230]:
from statsmodels.api import OLS
from statsmodels.tools.tools import add_constant
import pandas as pd
from scipy.stats import f

def chow_test(data, split_point, dependent_var, independent_vars):
    """
    Realiza o teste de Chow para verificar mudanças estruturais em um modelo de regressão.

    Args:
        data (pd.DataFrame): DataFrame contendo os dados.
        split_point (int): Ponto de divisão para o teste de Chow.
        dependent_var (str): Nome da variável dependente.
        independent_vars (list): Lista de nomes das variáveis independentes.

    Returns:
        dict: Resultados do teste de Chow contendo os valores F e p.
    """
    # Dividir os dados em dois subconjuntos
    data1 = data[data['periodo'] <= split_point]
    data2 = data[data['periodo'] > split_point]

    # Ajustar os modelos para cada subconjunto
    X1 = add_constant(data1[independent_vars])
    y1 = data1[dependent_var]
    model1 = OLS(y1, X1).fit()

    X2 = add_constant(data2[independent_vars])
    y2 = data2[dependent_var]
    model2 = OLS(y2, X2).fit()

    # Ajustar o modelo para o conjunto completo
    X_full = add_constant(data[independent_vars])
    y_full = data[dependent_var]
    model_full = OLS(y_full, X_full).fit()

    # Calcular os graus de liberdade
    k = len(independent_vars) + 1  # Número de parâmetros (incluindo o intercepto)
    n1 = len(data1)
    n2 = len(data2)
    n = n1 + n2

    # Calcular a soma dos quadrados dos resíduos
    RSS1 = model1.ssr
    RSS2 = model2.ssr
    RSS_full = model_full.ssr

    # Calcular o valor F
    F = ((RSS_full - (RSS1 + RSS2)) / k) / ((RSS1 + RSS2) / (n - 2 * k))

    # Calcular o p-valor
    p_value = 1 - f.cdf(F, k, n - 2 * k)

    return {"F-statistic": F, "p-value": p_value}


#####TODO: GOTTA REVIEW THIS
# Iterar pelas variáveis no main_regrdf
if ARGS['DoChowTest']:
    chow_results = {}
    split_point = POS_PANDEMIC_START_PERIOD_INDEX  # Exemplo: ponto de divisão para o teste de Chow
    dependent_var = DEPENDENT_VAR
    independent_vars = [INDEPENDENT_VAR] + [key for key in DUMMY_ARGS.keys()]

    for variable in main_regrdf[VARIABLE_NAME].unique():
        variable_data = main_regrdf[main_regrdf[VARIABLE_NAME] == variable]
        result = chow_test(variable_data, split_point, dependent_var, independent_vars)
        chow_results[variable] = result

    # Converter os resultados em um DataFrame para visualização
    chow_results_df = pd.DataFrame(chow_results).T
    chow_results_df.columns = ["F-statistic", "p-value"]
    print(chow_results_df)

In [231]:
model = pwlf.PiecewiseLinFit(main_regrdf[INDEPENDENT_VAR].values,main_regrdf[DEPENDENT_VAR].values)
knots = model.fit(2)
print(knots)
for knot in knots:
    print(find_date_from_idx(np.round(knot), PERIOD_DATE_MAP))

[ 1.         18.00000001 26.        ]
2016-03-31 00:00:00
2020-06-30 00:00:00
2022-06-30 00:00:00


In [232]:
import pandas as pd
import numpy as np
from statsmodels.stats.diagnostic import het_breuschpagan, acorr_ljungbox
from statsmodels.stats.stattools import durbin_watson
from scipy.stats import shapiro, normaltest, jarque_bera
import statsmodels.api as sm

def analyze_time_series(df: pd.DataFrame, variable_col: str, date_col: str, value_col: str):
    """
    Analyzes a melted DataFrame for heteroscedasticity, distribution, and autocorrelation.

    Parameters:
        df (pd.DataFrame): Melted DataFrame with columns for variable, date, and value.
        variable_col (str): Column name for the variable identifier.
        date_col (str): Column name for the date.
        value_col (str): Column name for the value.

    Returns:
        dict: A dictionary with analysis results for each variable.
    """
    results = {}

    for variable in df[variable_col].unique():
        # Filter data for the current variable
        variable_data = df[df[variable_col] == variable].sort_values(by=date_col)
        y = variable_data[value_col].values
        x = np.arange(len(y))  # Time index as independent variable

        # Add constant for regression
        X = sm.add_constant(x)

        # Fit OLS model
        model = sm.OLS(y, X).fit()

        # Heteroscedasticity tests
        bp_test = het_breuschpagan(model.resid, X)
        bp_pvalue = bp_test[1]

        # Normality tests
        shapiro_test = shapiro(model.resid)
        shapiro_pvalue = shapiro_test.pvalue

        jb_test = jarque_bera(model.resid)
        jb_pvalue = jb_test[1]

        dagostino_test = normaltest(model.resid)
        dagostino_pvalue = dagostino_test.pvalue

        # Autocorrelation tests
        dw_stat = durbin_watson(model.resid)
        ljungbox_test = acorr_ljungbox(model.resid, lags=[10], return_df=True)
        ljungbox_pvalue = ljungbox_test['lb_pvalue'].iloc[0]

        # Store results
        results[variable] = {
            "Heteroscedasticity (Breusch-Pagan)": bp_pvalue,
            "Normality (Shapiro-Wilk)": shapiro_pvalue,
            "Normality (Jarque-Bera)": jb_pvalue,
            "Normality (D'Agostino)": dagostino_pvalue,
            "Autocorrelation (Durbin-Watson)": dw_stat,
            "Autocorrelation (Ljung-Box)": ljungbox_pvalue,
        }

    return results

explorationresults = analyze_time_series(main_regrdf, VARIABLE_NAME, DATE_NAME, DEPENDENT_VAR)
explorationresultsDF = pd.DataFrame(explorationresults).T
explorationresultsDF

,Heteroscedasticity (Breusch-Pagan),Normality (Shapiro-Wilk),Normality (Jarque-Bera),Normality (D'Agostino),Autocorrelation (Durbin-Watson),Autocorrelation (Ljung-Box)
"(F00 - F09) Transtornos mentais orgânicos, inclusive os sintomáticos",0.027765,0.996193,8.730459e-01,8.114118e-01,0.893285,0.006813
(F10 - F19) Transtornos mentais e comportamentais devidos ao uso de substância psicoativa,0.202895,0.042347,2.822467e-01,2.283825e-01,0.709565,0.000927
"(F20 - F29) Esquizofrenia, transtornos esquizotípicos e transtornos delirantes",0.806404,0.001014,3.200268e-03,1.787202e-03,0.937058,0.251746
(F30 - F39) Transtornos do humor [afetivos],0.682506,0.021953,1.369225e-01,7.715194e-02,1.197910,0.255042
"(F40 - F48) Transtornos neuróticos, transtornos relacionados com o ""stress"" e transtornos somatoformes",0.020907,0.993702,9.358352e-01,9.092760e-01,0.768645,0.002344
(F50 - F59) Síndromes comportamentais associadas a disfunções fisiológicas e a fatores físicos,0.070307,0.619392,6.794046e-01,6.416143e-01,1.400063,0.016645
(F60 - F69) Transtornos da personalidade e do comportamento do adulto,0.124380,0.563494,7.680689e-01,6.283358e-01,0.698747,0.009415
(F70 - F79) Retardo mental,0.950025,0.013555,9.314806e-02,3.796808e-02,0.677464,0.008036
(F80 - F89) Transtornos do desenvolvimento psicológico,0.031533,0.044618,7.376570e-02,2.684301e-02,0.666307,0.000304
(F90 - F98) Transtornos do comportamento e transtornos emocionais que aparecem habitualmente durante a infância ou a adolescência,0.308719,0.881587,8.520141e-01,5.165640e-01,1.228735,0.410165


# Define Processors


In [233]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class RegressionProcessor:
    """
    Performs regression-related operations such as model fitting,
    best rho search, statistics extraction, and retrieving the real series.
    """

    def __init__(
        self,
        ARGS: Dict[str, Any],
        col_name: str,
        value_name: str,
        regression_df: pd.DataFrame,

    ):
        self.ARGS = ARGS
        self.col_name = col_name
        self.value_name = value_name
        # The DataFrame from which the "real series" will be obtained.
        # self.real_data_df = real_data_df
        self.regression_df = regression_df


        self.extra_data_storage = {"best_rho": None}

    ## called from other internal methods
    def get_variable_df(self, regression_df: pd.DataFrame, variable: str):
        return regression_df[regression_df[VARIABLE_NAME] == variable]
    def fit_arima(self, variable_df: pd.DataFrame, order: tuple):
        """
        Fits an ARIMA model to the time series data.

        Parameters:
            variable_df (pd.DataFrame): DataFrame containing the time series data for a variable.
            order (tuple): The (p, d, q) order of the ARIMA model.

        Returns:
            ARIMAResults: Fitted ARIMA model results.
        """
        y = variable_df[self.value_name].values
        model = ARIMA(y, order=order)
        results = model.fit()
        return results
    def model_fit(
        self, model, iterative_fit: bool = None, max_iterations: int = None, **kwARGS
    ):
        """
        Fits the model using iterative_fit if indicated.
        
        Parameters:
        -----------
        model : statsmodels model object
            The model to be fitted
        iterative_fit : bool, optional
            Whether to use iterative fitting
        max_iterations : int, optional
            Maximum iterations for iterative fitting
        kwARGS : dict
            Additional keyword arguments passed to fit()
            
        Returns:
        --------
        Fitted model results
        """
        # Get covariance parameters from instance ARGS
        cov_type = self.ARGS.get("CovType", "nonrobust")
        cov_kwds = self.ARGS.get("CovKwargs", {})
        
        # Handle default values for iterative fitting
        if iterative_fit is None:
            iterative_fit = self.ARGS.get("IterativeFit", False)
        if max_iterations is None:
            max_iterations = self.ARGS.get("MaxIterations", 100)
            
        # Prepare fit arguments including covariance specification
        fit_kwargs = {
            "cov_type": cov_type,
            "cov_kwds": cov_kwds,
            **kwARGS  # Merge additional user-provided kwargs
        }
        
        if iterative_fit:
            if hasattr(model, "iterative_fit"):
                fit_func = model.iterative_fit
                fit_kwargs["maxiter"] = max_iterations
            else:
                logger.warning(
                    "Model does not support iterative_fit. Using fit() instead."
                )
                fit_func = model.fit
        else:
            fit_func = model.fit
            
        return fit_func(**fit_kwargs)

    def find_best_rho(
        self,
        df: pd.DataFrame,
        formula: str,
        criterio: str = "aic",
        start: float = -1.0,
        stop: float = 1.01,
        step: float = 0.05,
    ) -> float:
        """
        Searches for the best rho parameter using the specified criterion.
        """
        best_rho = 1.0
        best_criterion_value = np.inf
        for rho in np.arange(start, stop, step):
            model = smf.glsar(formula, data=df, rho=rho)
            results = self.model_fit(model)
            criterion_value = getattr(results, criterio)
            if criterion_value < best_criterion_value:
                best_criterion_value = criterion_value
                best_rho = rho
        return best_rho
    '''
    def _durbin_watson(self, resid) -> float:
        """
        Computes Durbin-Watson statistic.
        """
        from statsmodels.stats.stattools import durbin_watson

        return durbin_watson(resid)
    '''
    def get_real_series(self, variable) -> pd.Series:
        """
        Retrieves the real data series for a variable.
        """
        return self.get_variable_df(self.regression_df, variable)
    '''
    def add_one_month(dt):  ### remover
        """
        Adds one month to a datetime object.

        ARGS:
                dt: The datetime object to modify.

        Returns:
                A new datetime object with one month added.
        """
        # Calculate the next month
        try:
            next_month = dt.replace(month=dt.month + 1)
        except ValueError:  # Handle cases like December
            next_month = dt.replace(year=dt.year + 1, month=1)

        return next_month
        ##########################
    '''
    def extract_result_stats(self, results) -> Dict[str, Any]:
        """
        Extracts regression parameters and returns them in a dictionary.
        """
        try:
            r_value = np.sqrt(results.rsquared)
        except (AttributeError, TypeError):
            r_value = np.nan

        try:
            converged = results.converged
        except Exception:
            converged = np.nan

        best_rho = self.extra_data_storage["best_rho"]
        #breakpoint()
        dummy_stats = {}
        dummy_vars = self.ARGS['DummyArgs'].keys() if self.ARGS['DummyArgs'] else []
        for var in dummy_vars:
            if var in results.params:
                dummy_stats[var] = {
                    "coef": results.params.get(var, np.nan),
                    "p_value": results.pvalues.get(var, np.nan),
                    'sgn' : results.pvalues.get(var, np.nan) < 0.05,  # At 5% significance
                    "conf_int": results.conf_int().loc[var].values.tolist(),
                    "std_err": results.bse.get(var, np.nan)
                }
            else:
                dummy_stats[var] = {
                    "coef": np.nan,
                    "p_value": np.nan,
                    "conf_int": [np.nan, np.nan],
                    "sgn" : False,
                    "std_err": np.nan
                }

        slope = results.params.get(self.col_name, np.nan)
        stats_dict = {
            "slope": slope,
            'dummy_stats': dummy_stats,
            #'slope_normalized': slope / np.mean(serie_values),
            "best_rho": best_rho,
            "intercept": results.params.get("Intercept", np.nan),
            "p_value": results.pvalues.get(self.col_name, np.nan),
            'sgnf': results.pvalues.get(self.col_name, np.nan) < 0.05,  # At 5% significance
            "std_err": results.bse.get(self.col_name, np.nan),
            "r_value": r_value,
            "r_squared": results.rsquared,
            "r_squared_adj": results.rsquared_adj,
            "Durbin-Watson": np.round(durbin_watson(results.resid), 4),
            "AIC": results.aic,
            "BIC": results.bic,
            "num_obs": results.nobs,
            "fvalue": results.fvalue,
            "f_pvalue": results.f_pvalue,
            "log_likelihood_f": results.llf,
            "condition_number": results.condition_number,
            "df_model": results.df_model,
            "df_resid": results.df_resid,
            "resid": results.resid,
            "bse": results.bse,
            "ssr": results.ssr,
            "centered_tss": results.centered_tss,
            "uncentered_tss": results.uncentered_tss,
            "ess": results.ess,
            "mse_model": results.mse_model,
            "mse_resid": results.mse_resid,
            "mse_total": results.mse_total,
            "tvalues": results.tvalues,
            "pvalues": results.pvalues,
            "converged": converged,
        }
        return stats_dict

    def extract_result_stats_arima(self, results) -> Dict[str, Any]:
        """
        Extracts ARIMA-specific regression parameters and returns them in a dictionary.
        """
        return results.summary()
        try:
            converged = results.mle_retvals.get("converged", np.nan)
        except AttributeError:
            converged = np.nan
    
        stats_dict = {
            "aic": results.aic,
            "bic": results.bic,
            "hqic": results.hqic,
            "log_likelihood": results.llf,
            "converged": converged,
            "params": results.params.to_dict(),
            "stderr": results.bse.to_dict(),
        }
        return stats_dict
    def get_regression_results(self, variable: str):
        """
        Returns regression results based on the specified method in ARGS.
        Supports OLS, Prais-Winsten, and ARIMA.
        """
        method = self.ARGS.get("RegressionMethod", "ols")
        formula = self.ARGS.get("ModelFormula", "")
        variable_df = self.get_variable_df(self.regression_df, variable)

        if method == "prais_winsten":
            best_rho = self.find_best_rho(
                variable_df, formula, criterio=self.ARGS.get("CriterioBestRho", "aic"),
                start=self.ARGS.get('best_rho_range', None)[0],
                stop=self.ARGS.get('best_rho_range', None)[1],
                step=self.ARGS.get('best_rho_range', None)[2],
            )
            self.extra_data_storage["best_rho"] = best_rho
            model = smf.glsar(formula, data=variable_df, rho=best_rho)
            results = self.model_fit(model)
        elif method == "arima":
            arima_order = self.ARGS.get("ARIMAOrder", (1, 1, 1))  # Default to (1, 1, 1)
            results = self.fit_arima(variable_df, order=arima_order)
            print(results.summary())
        else:  # Default to OLS
            model = smf.ols(formula, data=variable_df)
            results = self.model_fit(model)

        return results
    '''
    def get_dummy_impact(
        self, variable: str, results
    ):
        if results is not None:
            dummy_vars = self.ARGS.get("DummyArgs", {}).keys()
            # Extract coefficients, p-values, and confidence intervals for dummies
            dummy_impact = {}
            for var in dummy_vars:
                if var in results.params:
                    coef = results.params[var]
                    p_value = results.pvalues[var]
                    conf_int = results.conf_int().loc[var].values.tolist()
                    dummy_impact[var] = {
                        "coef": coef,
                        "p_value": p_value,
                        "conf_int": conf_int,
                        "significant": p_value < 0.05,  # At 5% significance
                    }
            
            return dummy_impact
                # Store or return dummy impact
                # self.extra_data_storage['dummy_impact'] = dummy_impact
    '''
    def get_regression_prediction_data(
        self, variable: str, results_params: any
    ) -> pd.Series:
        """
        Generates a prediction series for a variable based on the regression results.

        ARGS:
                variable_df (pd.DataFrame): The DataFrame used to obtain periods (columns).
                results_params: Either a dictionary of regression parameters (with keys 'slope' and 'intercept')
                                or a RegressionResultsWrapper from which these values can be extracted.

        Returns:
                pd.Series: The predicted values with an extended period index.
        """
        ######
        if self.ARGS['RegressionMethod'] == 'arima':
            return results_params.forecast(steps=30)


        # If results_params is not a dict, try to extract the needed values from its attributes.
        if not isinstance(results_params, dict):
            slope = results_params.params.get(self.col_name, np.nan)
            intercept = results_params.params.get("Intercept", np.nan)
        else:
            slope = results_params.get("slope", np.nan)
            intercept = results_params.get("intercept", np.nan)

        variable_df = self.get_variable_df(self.regression_df, variable)

        # Extract periods and dates.
        original_periods = variable_df[INDEPENDENT_VAR].tolist()
        original_dates = pd.to_datetime(variable_df[DATE_NAME]).tolist()

        last_period = original_periods[-1]
        last_date = original_dates[-1]

        # Determine number of additional future points to predict.
        num_additional = int(
            len(original_periods) * self.ARGS["additional_dates_factor"]
        )

        # Generate future periods and dates.
        additional_periods = [last_period + i for i in range(1, num_additional + 1)]
        additional_dates = [
            last_date + relativedelta(months=i * self.ARGS["PeriodGrouping"])
            for i in range(1, num_additional + 1)
        ]

        # Combine original + additional.
        all_periods = original_periods + additional_periods
        all_dates = original_dates + additional_dates

        # Compute predictions using periods.
        predicted_values = [slope * period + intercept for period in all_periods]

        # Build the prediction series with dates as index.
        prediction_series = pd.Series(
            predicted_values, index=pd.Index(all_dates), name=variable
        )
        return prediction_series

In [234]:
class SheetExporter:
    """
    Handles exporting Pandas DataFrames to Google Sheets (Colab) or saving locally (non-Colab).
    """

    def __init__(self, output_wksheet=None, output_path: str = LOCAL_OUTPUT_PATH):
        """
        :param output_wksheet: gspread.Spreadsheet object (used in Colab)
        :param output_path: Path to save file locally (used outside Colab)
        """
        self.output_wksheet = output_wksheet
        self.output_path = output_path

    def try_create_worksheet(self, title: str) -> None:
        if self.output_wksheet:
            try:
                self.output_wksheet.worksheet(title)
            except gspread.exceptions.WorksheetNotFound:
                self.output_wksheet.add_worksheet(title=title, rows="100", cols="20")
                logger.info(f"Worksheet '{title}' created.")

    def convert_int64_for_json(self, input_df: pd.DataFrame) -> pd.DataFrame:
        df = input_df.copy()
        for column in df.columns:
            if df[column].dtype == "int64":
                df[column] = df[column].astype(int)
            elif df[column].apply(lambda x: isinstance(x, np.int64)).any():
                df[column] = df[column].apply(
                    lambda x: int(x) if isinstance(x, np.int64) else x
                )
        return df

    def export_to_sheet(
        self, df: pd.DataFrame, title: str, include_index: bool = True
    ) -> None:
        clean_df = self.convert_int64_for_json(df)

        if self.output_wksheet:
            self.try_create_worksheet(title)
            ws = self.output_wksheet.worksheet(title)
            ws.clear()
            set_with_dataframe(ws, clean_df, include_index=include_index)
            logger.info(f"DataFrame exported to sheet '{title}'.")

        elif self.output_path:
            # Check if file exists
            file_exists = os.path.exists(self.output_path)

            # Attempt to open file and catch PermissionError if the file is open elsewhere
            if file_exists:
                while True:
                    try:
                        with pd.ExcelWriter(self.output_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
                            clean_df.to_excel(writer, sheet_name=title, index=include_index)
                        break  # Exit the loop if writing succeeds
                    except PermissionError:
                        logger.warning(f"File '{self.output_path}' is currently open. Please close the file and press Enter to try again.")
                        input("Press Enter after closing the file...")
                        time.sleep(5)  # Optional: wait before retrying
            else:
                # Create a new workbook and write the sheet
                with pd.ExcelWriter(self.output_path, engine='openpyxl') as writer:
                    clean_df.to_excel(writer, sheet_name=title, index=include_index)

        else:
            logger.error(
                "No output method specified. Please provide either a gspread worksheet or a local path."
            )

In [235]:
class ResultsDataProcessor:
    """
    Coordinates the regression processing and exporting.
    It can append regression data from a DataFrame (e.g. RegressionDataDF),
    build exportable DataFrames using configuration, and export to Google Sheets.
    """

    def __init__(
        self,
        ARGS: Dict[str, Any],
        sheet_exporter: SheetExporter,
        regression_processor: RegressionProcessor,
        is_total = False,
        
    ):
        self.ARGS = ARGS

        self.data_dict: Dict[str, Dict[str, Any]] = {}  # per-variable results storage

        self.sheet_exporter = sheet_exporter
        self.regression_processor = regression_processor

        self.is_total = is_total

        # Configuration for building exportable DataFrames.
        # Keys in export_config refer to DataFrame names that will be built.
        if True:#self.is_total:
            self.export_config: Dict[str, Dict[str, Any]] = {
                "LinearRegressionResultStats": {
                    "transform": lambda var, data: pd.DataFrame(data["stats"], index=[var]),
                    "include_index": True,
                },

                "DummyImpact": {
                    "transform": lambda var, data: self.get_dummy_df(
                        var, data["stats"]['dummy_stats']),
                    "include_index": True,
                },

                #### TODO: EXPORT THE PERIOD AS WELL?
                "RealDataSeries": {
                    "transform": lambda var, data: self.create_serie_df_test(
                        data["real_serie"]
                    ),
                    "include_index": True,
                },
                "PredictionSeries": {
                    "transform": lambda var, data: self.create_serie_df(
                        data["prediction_serie"]
                    ),
                    "include_index": True,
                },
            }
        else:
            self.export_config = {}
        # Configuration for appending per-variable results.
        # Keys here are produced for each variable and then later used in export.
        self.variable_config: Dict[str, Any] = {
            ### A dictionary with RegressionWrapper objecs
            "results": lambda var: self.regression_processor.get_regression_results(
                var,
            ),


            "stats": lambda var: (
                self.regression_processor.extract_result_stats_arima(self.data_dict[var]["results"])
                if self.ARGS.get("RegressionMethod", "ols") == "arima"
                else self.regression_processor.extract_result_stats(self.data_dict[var]["results"])
            ),
            "real_serie": lambda var: self.regression_processor.get_real_series(var),
            "prediction_serie": lambda var: self.regression_processor.get_regression_prediction_data(
                var, self.data_dict[var]["results"]  # ✅ Re-use stored results
            ),
        }

    def process_data(
        self,
    ):  ### this hould iterate the self.variable_config, previoulsy was 'append_regression_data_df'

        for variable in self.regression_processor.regression_df[VARIABLE_NAME].unique():
            self.data_dict[variable] = {}
            for key, func in self.variable_config.items():
                self.data_dict[variable][key] = func(
                    variable
                )  ## retrieves a function from the variable_config and executes it in a by variable basis, stores it
            logger.info(f"Processed results for variable '{variable}'.")

    def convert_ordinal_to_datetime(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Converts DataFrame columns from numeric date format to datetime.
        Using origin as '1899-12-30' for excel serial dates.
        Returns a new DataFrame with renamed columns.
        """
        # Check if columns are numeric (likely ordinal)
        if all(isinstance(col, (int, np.integer)) for col in df.columns):
            new_columns = df.columns.map(lambda x: datetime.fromordinal(x))
            renamed_df = df.copy()
            renamed_df.columns = new_columns
            return renamed_df
        else:
            # Columns are not numeric (likely already datetime)
            return df

    ### TODO: has to be changed, it now receives a DF (melted). This function should shape the df for sheet exportion
    def create_serie_df(self, serie: pd.Series) -> pd.DataFrame:

        df = pd.DataFrame(serie).T
        df.index = [serie.name]
        return df

    def create_serie_df_test(self, df: pd.DataFrame) -> pd.DataFrame:
        shaped_df = df.pivot(index=VARIABLE_NAME, columns=DATE_NAME, values=DEPENDENT_VAR)

        # Optional: If you want to have a sorted order on the columns (dates)
        shaped_df = shaped_df.reindex(sorted(shaped_df.columns), axis=1)

        return shaped_df
    '''
    def get_dummy_df(self, variable:str, data):
        stats = list(next(iter(data.values())).keys())  
        # Build rows for each stat
        rows = []
        for stat in stats:
            row = {"var": variable, "stat": stat}
            for dummy_var, values in data.items():
                row[dummy_var] = values.get(stat, np.nan)
            rows.append(row)    
        # Convert to DataFrame
        return pd.DataFrame(rows)
    '''
    def get_dummy_df(self, variable: str, data: dict) -> pd.DataFrame:
        """
        Transforma dicionário de estatísticas em DataFrame "wide" com nomes de colunas como:
        coef_Pandemia, p_value_Pandemia, etc.
        """

        # Extrair todas as estatísticas (assumindo que todas têm o mesmo conjunto)
        stats = list(next(iter(data.values())).keys())

        # Inicializa um dicionário para a linha final
        row = {}

        # Popula com chaves no formato "stat_period"
        for period, stat_dict in data.items():
            for stat in stats:
                col_name = f"{period}___{stat}"
                row[col_name] = stat_dict.get(stat, np.nan)

        # Cria DataFrame com a variável como índice
        df = pd.DataFrame([row], index=[variable])
        return df

    def get_exportable_dfs(self) -> Dict[str, pd.DataFrame]:
        """
        Builds a dictionary of DataFrames ready for export based on export_config.
        """
        export_dfs: Dict[str, pd.DataFrame] = {}
        for df_name, config in self.export_config.items():
            dfs = []
            for variable, data in self.data_dict.items():
                dfs.append(config["transform"](variable, data))
            export_dfs[df_name] = pd.concat(dfs)

        # Add metadata (if any)
        meta_df = pd.DataFrame(list(self.ARGS.items()), columns=["key", "value"])
        export_dfs["Metadata"] = meta_df
        return export_dfs

    def process_and_export(self, is_total :bool = False) -> Dict[str, pd.DataFrame]:
        """
        Processes data into exportable DataFrames and exports them to Google Sheets.
        Returns the dictionary of DataFrames.
        """
        export_dfs = self.get_exportable_dfs()
        for sheet_title, df in export_dfs.items():
            include_index = self.export_config.get(sheet_title, {}).get(
                "include_index", True
            )
            if is_total:
                 self.sheet_exporter.export_to_sheet(
                df, sheet_title, include_index=include_index
                )
        return export_dfs

# **EXEC** --- Instantiate Processor and Process/Export

In [236]:
def getRegressionResults(df: pd.DataFrame, title: str, export_wksheet=output_wksheet):
    # Instantiate the SheetExporter using your output worksheet
    is_total = title == 'Total'
    sheet_exporter = SheetExporter(output_wksheet)

    # Instantiate the RegressionProcessor with necessary parameters
    regression_processor = RegressionProcessor(ARGS, INDEPENDENT_VAR, DEPENDENT_VAR, df)

    # Instantiate the ResultsDataProcessor; it will coordinate between the modules.
    processor = ResultsDataProcessor(ARGS, sheet_exporter, regression_processor, is_total)

    # Feed the entire regression data DataFrame into the processor
    processor.process_data()

    # Process data and export it to Google Sheets; also retrieve exportable DataFrames for further analysis.

    exported_dfs = processor.process_and_export(is_total=is_total)
    logger.info("Processing and export completed.")
    return exported_dfs


In [237]:
RegressionResultsTable = {}

for title, df in RegressionDfsTable.items():
    RegressionResultsTable[title] = getRegressionResults(df, title)

INFO:__main__:Processed results for variable '(F00 - F09) Transtornos mentais orgânicos, inclusive os sintomáticos'.
INFO:__main__:Processed results for variable '(F10 - F19) Transtornos mentais e comportamentais devidos ao uso de substância psicoativa'.
INFO:__main__:Processed results for variable '(F20 - F29) Esquizofrenia, transtornos esquizotípicos e transtornos delirantes'.
INFO:__main__:Processed results for variable '(F30 - F39) Transtornos do humor [afetivos]'.
INFO:__main__:Processed results for variable '(F40 - F48) Transtornos neuróticos, transtornos relacionados com o "stress" e transtornos somatoformes'.
INFO:__main__:Processed results for variable '(F50 - F59) Síndromes comportamentais associadas a disfunções fisiológicas e a fatores físicos'.
INFO:__main__:Processed results for variable '(F60 - F69) Transtornos da personalidade e do comportamento do adulto'.
INFO:__main__:Processed results for variable '(F70 - F79) Retardo mental'.
INFO:__main__:Processed results for var

In [238]:
# Create a dict to hold {variable: {title: slope}} structure
def visualize_slopes(regression_results_table = RegressionResultsTable):
    slope_dict = {}

    for title, dfs in regression_results_table.items():
        slopes = dfs["LinearRegressionResultStats"]["slope"]
        for variable, slope in slopes.items():
            if variable not in slope_dict:
                slope_dict[variable] = {}
            slope_dict[variable][title] = slope

    # Convert to DataFrame
    table_df = pd.DataFrame(slope_dict).T  # Transpose so variables are rows
    table_df.index.name = "Variável"
    table_df.columns.name = "Título"

    # Optional: sort rows by slope under a specific title
    table_df = table_df.sort_values(by=table_df.columns[0], ascending=False)
    return table_df

#SlopesTable = visualize_slopes()
#SlopesTable

# Define Hover Graph Func


In [239]:
import plotly.graph_objects as go


def interactive_plot_real_vs_prediction_series(
    real_df: pd.DataFrame,
    pred_df: pd.DataFrame,
    std_err_data=None,
    highlight_period="2020",
):
    """
    Plots real vs. prediction time series with hover data using Plotly,
    creating separate plots for each variable.
    """
    # Make copies
    real_copy = real_df.copy()
    pred_copy = pred_df.copy()
    # Align shapes by extending the smaller DataFrame
    if pred_copy.shape[1] < real_copy.shape[1]:
        missing_cols = real_copy.columns.difference(pred_copy.columns)
        for col in missing_cols:
            pred_copy[col] = np.nan
        pred_copy = pred_copy[real_copy.columns]
    elif real_copy.shape[1] < pred_copy.shape[1]:
        missing_cols = pred_copy.columns.difference(real_copy.columns)
        for col in missing_cols:
            real_copy[col] = np.nan
        real_copy = real_copy[pred_copy.columns]
        # Create separate figures for each variable
    for idx in real_copy.index:
        fig = go.Figure()

        fig.add_trace(
            go.Scatter(
                x=real_copy.columns,
                y=real_copy.loc[idx],
                mode="markers",  # use only 'markers' for scatter
                name="Real",
                hovertemplate="<b>%{x}</b><br>Real: %{y:.2f}<extra></extra>",  # Updated hovertemplate
            )
        )
        # Line plot for prediction data with error bars (if std_err_data is provided)
        if std_err_data is not None and idx in std_err_data:
            fig.add_trace(
                go.Scatter(
                    x=pred_copy.columns,
                    y=pred_copy.loc[idx],
                    mode="lines+markers",
                    name="Prediction",
                    hovertemplate="<b>%{x}</b><br>Prediction: %{y:.2f}<extra></extra>",
                    error_y=dict(
                        type="data",
                        array=[std_err_data[idx]]
                        * len(pred_copy.columns),  # Convert to list for error_y
                        visible=True,
                        thickness=1.5,  # Adjust thickness for visibility
                        width=6,  # Adjust width for visibility
                    ),
                )
            )
        else:
            # If no standard error data, plot without error bars
            fig.add_trace(
                go.Scatter(
                    x=pred_copy.columns,
                    y=pred_copy.loc[idx],
                    mode="lines+markers",
                    name="Prediction",
                    hovertemplate="<b>%{x}</b><br>Prediction: %{y:.2f}<extra></extra>",
                )
            )

        if highlight_period:
            fig.update_layout(
                shapes=[
                    dict(
                        type="rect",
                        xref="x",
                        yref="paper",
                        x0=real_copy.columns[
                            real_copy.columns.astype(str).str.startswith(
                                highlight_period
                            )
                        ][0],
                        y0=0,
                        x1=real_copy.columns[
                            real_copy.columns.astype(str).str.startswith(
                                highlight_period
                            )
                        ][-1],
                        y1=1,
                        fillcolor="LightSalmon",
                        opacity=0.5,
                        line_width=0,
                        layer="below",
                    )
                ]
            )

        fig.update_layout(
            title=f"Variable: {idx}", xaxis_title="Time", yaxis_title="Value"
        )
        fig.show()
        # fig.update_layout(title=f'Variable: {idx}', xaxis_title='Time', yaxis_title='Value')
        # fig.show()

    return real_copy, pred_copy

In [240]:
import numpy as np
import pandas as pd
from datetime import datetime
import plotly.graph_objects as go
from sklearn.base import BaseEstimator, TransformerMixin


def interactive_plot_multi_regression_series(
    regression_tables,
    highlight_period : str | None = "2020",
    #real_df: pd.DataFrame,
    #pred_dfs: dict[str, pd.DataFrame],
    #std_err_data: dict[str, dict] | None = None,
    #highlight_period: str | None = "2020",
):
    """
    Plot real vs. multiple prediction time series with hover, error bars, and optional highlight.

    Parameters
    ----------
    real_df
        DataFrame of “ground truth” series (index = series IDs, columns = datetime-like).
    pred_dfs
        Dict mapping a name → prediction DataFrame to plot as a separate line.
    std_err_data
        Optional dict mapping the same names → dict of {series ID: std_error}.
    highlight_period
        A string prefix (e.g. '2020') to highlight columns whose str(column) startswith this.
    """
    # 1) Copy and parse all column axes to datetime if possible
    pred_dfs = {}
    real_copy = None
    std_errs = {}
    for title, dfs in regression_tables.items():
        pred_dfs[title] = dfs['PredictionSeries'].copy()
        std_errs[title] = dfs['LinearRegressionResultStats']['std_err'].copy()
        if real_copy is None or len(dfs['RealDataSeries'].columns) > len(real_copy.columns):
            real_copy = dfs['RealDataSeries'].copy()
    ####

    try:
        real_copy.columns = pd.to_datetime(real_copy.columns)
    except:
        pass

    # 2) Build a unified column index across real + all preds
    all_cols = real_copy.columns
    for df in pred_dfs.values():
        cols = df.columns
        try:
            cols = pd.to_datetime(cols)
        except:
            pass
        all_cols = all_cols.union(cols)
    all_cols = all_cols.sort_values()

    # 3) Reindex real + each pred onto that same axis
    real_copy = real_copy.reindex(columns=all_cols).sort_index(axis=1)
    pred_copies = {
        name: (
            df.copy()
            .pipe(
                lambda d: (
                    d.set_axis(pd.to_datetime(d.columns), axis=1, copy=False)
                    if not pd.api.types.is_datetime64_any_dtype(d.columns)
                    else d
                )
            )
            .reindex(columns=all_cols)
            .sort_index(axis=1)
        )
        for name, df in pred_dfs.items()
    }

    # 4) Loop over each series in real_copy
    for series_id in real_copy.index:
        fig = go.Figure()

        # 4a) Plot the real data
        fig.add_trace(
            go.Scatter(
                x=all_cols,
                y=real_copy.loc[series_id],
                mode="markers",
                name="Real",
                hovertemplate="<b>%{x|%Y-%m-%d}</b><br>Real: %{y:.2f}<extra></extra>",
            )
        )

        # 4b) Plot each prediction line (with optional error bars)
        for name, pred_df in pred_copies.items():
            y_pred = pred_df.loc[series_id]
            trace_kwARGS = dict(
                x=all_cols,
                y=y_pred,
                mode="lines+markers",
                name=name,
                hovertemplate=f"<b>%{{x|%Y-%m-%d}}</b><br>{name}: %{{y:.2f}}<extra></extra>",
            )

            # add error_y if provided for this line & series
            if (
                std_errs[name][series_id]
            ):
                err_val = std_errs[name][series_id]
                trace_kwARGS["error_y"] = dict(
                    type="data",
                    array=[err_val] * len(all_cols),
                    visible=True,
                    thickness=1.5,
                    width=6,
                )

            fig.add_trace(go.Scatter(**trace_kwARGS))

        # 4c) Highlight a “period” if requested
        if highlight_period:
            # find all columns whose string repr startswith highlight_period
            mask = [str(col).startswith(highlight_period) for col in all_cols]
            if any(mask):
                x0 = all_cols[mask][0]
                x1 = all_cols[mask][-1]
                fig.add_shape(
                    type="rect",
                    xref="x",
                    yref="paper",
                    x0=x0,
                    y0=0,
                    x1=x1,
                    y1=1,
                    fillcolor="LightSalmon",
                    opacity=0.3,
                    line_width=0,
                    layer="below",
                )

        fig.update_layout(
            title=f"Series: {series_id}",
            xaxis_title="Time",
            yaxis_title="Value",
            legend=dict(title="Legend"),
        )
        fig.show()

    # Return the aligned copies in case you need them
    return real_copy, pred_copies




In [241]:
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose

def interactive_plot_with_decomposition(
	regression_tables,
	highlight_period: str | None = "2020",
	freq: str = "MS",  # Monthly Start default
	model: str = "additive",  # or "multiplicative"
):
	pred_dfs = {}
	real_copy = None
	std_errs = {}
	for title, dfs in regression_tables.items():
		pred_dfs[title] = dfs['PredictionSeries'].copy()
		std_errs[title] = dfs['LinearRegressionResultStats']['std_err'].copy()
		if real_copy is None or len(dfs['RealDataSeries'].columns) > len(real_copy.columns):
			real_copy = dfs['RealDataSeries'].copy()

	# Standardize column dates
	real_copy.columns = pd.to_datetime(real_copy.columns, errors='coerce')
	all_cols = real_copy.columns
	for df in pred_dfs.values():
		df.columns = pd.to_datetime(df.columns, errors='coerce')
		all_cols = all_cols.union(df.columns)
	all_cols = all_cols.sort_values()

	# Reindex and align
	real_copy = real_copy.reindex(columns=all_cols)
	pred_copies = {
		name: df.reindex(columns=all_cols)
		for name, df in pred_dfs.items()
	}

	# Plot per series
	for series_id in real_copy.index:
		fig = make_subplots(
			rows=4, cols=1,
			shared_xaxes=True,
			vertical_spacing=0.02,
			subplot_titles=("Prediction vs. Real", "Trend", "Seasonal", "Residual"),
		)

		# Real
		fig.add_trace(
			go.Scatter(
				x=all_cols,
				y=real_copy.loc[series_id],
				mode="markers",
				name="Real",
				hovertemplate="<b>%{x|%Y-%m-%d}</b><br>Real: %{y:.2f}<extra></extra>",
			),
			row=1, col=1
		)

		# Predictions
		for name, pred_df in pred_copies.items():
			trace_kw = dict(
				x=all_cols,
				y=pred_df.loc[series_id],
				mode="lines+markers",
				name=name,
				hovertemplate=f"<b>%{{x|%Y-%m-%d}}</b><br>{name}: %{{y:.2f}}<extra></extra>",
			)
			if std_errs[name][series_id]:
				trace_kw["error_y"] = dict(
					type="data",
					array=[std_errs[name][series_id]] * len(all_cols),
					visible=True,
				)
			fig.add_trace(go.Scatter(**trace_kw), row=1, col=1)

		# Decompose
		try:
			series = real_copy.loc[series_id]
			series.index = pd.DatetimeIndex(all_cols)
			series = series.dropna()
			series = series.asfreq(freq)

			result = seasonal_decompose(series, model=model)
			fig.add_trace(go.Scatter(x=series.index, y=result.trend, name="Trend"), row=2, col=1)
			fig.add_trace(go.Scatter(x=series.index, y=result.seasonal, name="Seasonal"), row=3, col=1)
			fig.add_trace(go.Scatter(x=series.index, y=result.resid, name="Residual"), row=4, col=1)

		except Exception as e:
			print(f"Could not decompose series {series_id}: {e}")

		# Highlight (row 1 only)
		if highlight_period:
			mask = [str(col).startswith(highlight_period) for col in all_cols]
			if any(mask):
				x0 = all_cols[mask][0]
				x1 = all_cols[mask][-1]
				fig.add_shape(
					type="rect",
					xref="x",
					yref="paper",
					x0=x0,
					x1=x1,
					y0=0,
					y1=0.25,
					fillcolor="LightSalmon",
					opacity=0.3,
					line_width=0,
					row=1, col=1
				)

		fig.update_layout(
			height=1000,
			title_text=f"Series {series_id} — Regression + Decomposition",
			showlegend=True
		)
		fig.show()


In [242]:
def plot_all_variables(dates, real_data, regression_stats, dummy_stats, pandemic_start_idx, pandemic_end_idx, trend_start_idx):
    """
    Iterates over variables and applies the plot_with_dummy_components function.

    Args:
        dates: Array of T datetimes.
        real_data: DataFrame where each row corresponds to a variable's real series.
        regression_stats: DataFrame containing regression statistics (e.g., intercept, slope).
        dummy_stats: DataFrame containing dummy variable coefficients (e.g., pulse_coef, slope_change_coef).
        pandemic_start_idx: Integer index where the pandemic starts (0-based).
    """
    for variable in real_data.index:
        # Extract data for the current variable
        real_series = real_data.loc[variable].values
        intercept = regression_stats.loc[variable, 'intercept']
        slope = regression_stats.loc[variable, 'slope']
        slope_change_coef = dummy_stats.loc[variable, 'Trend_PandemiaePos___coef']
        pulse_coef = dummy_stats.loc[variable, 'Pulse_Pandemia___coef']

        # Call the plot function for the current variable
        plot_with_dummy_components(
            dates=dates,
            real=real_series,
            intercept=intercept,
            slope=slope,
            slope_change_coef=slope_change_coef,
            pulse_coef=pulse_coef,
            step_start_idx=pandemic_start_idx,
            step_end_idx =pandemic_end_idx,
            trend_start_idx=trend_start_idx,
            pulse_duration = 1 if ARGS['FrequencyDateGroup'] == "Q" or ARGS['FrequencyDateGroup'] == "QE" else 6,
            variable_name=variable,
        )


def plot_with_dummy_components(
    dates,                    # array of T datetimes
    real,                     # real series, length T
    intercept,                # α
    slope,                    # β
    slope_change_coef,        # γ
    pulse_coef,               # ε (new argument for pulse effect)
    step_start_idx,           # t0: integer index where pandemic starts (0‑based)
    step_end_idx,             # t2: integer index where pandemic ends (0‑based)
    trend_start_idx,           # t1: integer index where trend change starts (0‑based)
    pulse_duration=1,         # Number of periods the pulse effect lasts
    variable_name=None,       # optional variable name for title
        ):
    # 1) Build numeric time index
    t = np.arange(len(dates)) + 1                     # 1,2,...,T

    # 2) Build pulse dummy (active for multiple periods starting at the pandemic start index)
    D_pulse = ((t >= (step_start_idx + 1)) & (t <= (step_end_idx+1))).astype(int)

    # 3) Time-since-start (t - t0) but zero before
    time_since = np.where(t > (trend_start_idx + 1), t - (trend_start_idx + 1), 0)

    # 4) Compute components
    baseline = intercept + slope * t
    pulse_only = baseline + pulse_coef * D_pulse
    full_pred = pulse_only + slope_change_coef * time_since

    # 5) Plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dates, y=real,
                             mode='markers', name='Real'))
    fig.add_trace(go.Scatter(x=dates, y=baseline,
                             mode='lines', name='Baseline',
                             line=dict(dash='dot')))
    fig.add_trace(go.Scatter(x=dates, y=full_pred,
                             mode='lines', name='Full model'))

    # Highlight pandemic start
    start_date = dates[step_start_idx]
    fig.add_vline(x=start_date, line_dash='longdash', line_color='grey')
    fig.add_annotation(x=start_date, y=max(real.max(), full_pred.max()),
                       text="Pandemic start", showarrow=True)
    
        # Add shaded background for the pulse period
    pulse_start_date = dates[step_start_idx]
    pulse_end_date = dates[step_end_idx]#dates[min(step_start_idx + pulse_duration, len(dates) - 1)]
    fig.add_shape(
        type="rect",
        x0=pulse_start_date,
        x1=pulse_end_date,
        y0=0,
        y1=1,
        xref="x",
        yref="paper",
        fillcolor="LightSalmon",
        opacity=0.3,
        layer="below",
        line_width=0,
    )

    fig.update_layout(title=f"{variable_name} - Decomposition of Regression Fit",
                      xaxis_title="Time",
                      yaxis_title="Value")
    fig.show()

# PLOT --- Hover Graph


In [243]:
regression = RegressionResultsTable['Total']
toplot_stats = regression['LinearRegressionResultStats']
real_serie = regression['RealDataSeries']
dummy_stats = regression['DummyImpact']


TODO: IMPORTANT!!!! Review the process of drawing the full model line!

TODO: Fazer slope em relação a media


fevereiro, segundo onda?

In [244]:
plot_all_variables(
    dates=real_serie.columns,
    real_data=real_serie,
    regression_stats=toplot_stats,
    dummy_stats=dummy_stats,
    pandemic_start_idx=PANDEMIC_START_PERIOD_INDEX,
    pandemic_end_idx=PANDEMIC_END_PERIOD_INDEX,  # Assuming pandemic ends at the same time as the data ends
    trend_start_idx=POS_PANDEMIC_START_PERIOD_INDEX,  # Assuming trend change starts at the same time as the pandemic
)



In [245]:
'''
regression = RegressionResultsTable['Total']
toplot_stats = regression['LinearRegressionResultStats']
real_serie = regression['RealDataSeries']
dummy_stats = regression['DummyImpact']

#'Pulse_Pandemia': {
#'Step_PandemiaePos'
#'Trend_PandemiaePos
pandemia_start_date = ARGS['DummyArgs']['Step_PandemiaePos']['start_date']


plot_with_dummy_components(real_serie.columns,
                           
    real_serie.values,
    toplot_stats['intercept'],
    toplot_stats['slope'],
    dummy_stats['Step_PandemiaePos___coef'],
    dummy_stats['Trend_PandemiaePos___coef'],
    17)
 '''

"\nregression = RegressionResultsTable['Total']\ntoplot_stats = regression['LinearRegressionResultStats']\nreal_serie = regression['RealDataSeries']\ndummy_stats = regression['DummyImpact']\n\n#'Pulse_Pandemia': {\n#'Step_PandemiaePos'\n#'Trend_PandemiaePos\npandemia_start_date = ARGS['DummyArgs']['Step_PandemiaePos']['start_date']\n\n\nplot_with_dummy_components(real_serie.columns,\n                           \n    real_serie.values,\n    toplot_stats['intercept'],\n    toplot_stats['slope'],\n    dummy_stats['Step_PandemiaePos___coef'],\n    dummy_stats['Trend_PandemiaePos___coef'],\n    17)\n "

In [246]:

interactive_plot_with_decomposition(RegressionResultsTable, highlight_period="2020", freq=ARGS['FrequencyDateGroup'], model="additive")

In [247]:
#real_copy, pred_copies = interactive_plot_multi_regression_series(RegressionResultsTable, highlight_period="2020")

# ### Old trash


In [248]:
def plot_real_vs_prediction_series(real_df: pd.DataFrame, pred_df: pd.DataFrame):
    """
    Plots real vs. prediction time series for each variable (row).
    Non-destructive: creates internal copies and extends the shorter one with NaNs to match.
    """
    # Make copies
    real_copy = real_df.copy()
    pred_copy = pred_df.copy()

    # Align shapes by extending the smaller DataFrame
    if pred_copy.shape[1] < real_copy.shape[1]:
        missing_cols = real_copy.columns.difference(pred_copy.columns)
        for col in missing_cols:
            pred_copy[col] = np.nan
        pred_copy = pred_copy[real_copy.columns]
    elif real_copy.shape[1] < pred_copy.shape[1]:
        missing_cols = pred_copy.columns.difference(real_copy.columns)
        for col in missing_cols:
            real_copy[col] = np.nan
        real_copy = real_copy[pred_copy.columns]

    # Plot each variable (row)
    for idx in real_copy.index:
        plt.figure(figsize=(10, 4))
        plt.plot(
            real_copy.columns,
            real_copy.loc[idx],
            label="Real",
            linestyle="-",
            marker="o",
        )
        plt.plot(
            pred_copy.columns,
            pred_copy.loc[idx],
            label="Prediction",
            linestyle="--",
            marker="x",
        )
        plt.title(f"Variable: {idx}")
        plt.xlabel("Time")
        plt.ylabel("Value")
        plt.legend()
        # plt.grid(True)
        plt.grid(True, which="both", linestyle="--", linewidth=0.5)
        # plt.xticks(real_copy.columns, rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

    return real_copy, pred_copy

In [249]:
#real_copy, pred_copy = plot_real_vs_prediction_series(
#    ExportedDFs["RealDataSeries"], ExportedDFs["PredictionSeries"]
#)

In [250]:
### TCR stuff
PeriodosExcluirAnalise = list(
    set(
        [
            (period - 1) // ARGS["PeriodGrouping"] + 1
            for period in PeriodosExcluirAnalise_original
        ]
    )
)


def CalcSigCI_RealvsPredicted(
    prediction_data,
    regression_results_stats,
    variable_values,
    lista_anos_de_analise=PeriodosExcluirAnalise,
    deg_freedom=1,
    CI=0.975,
):
    CI_data_dict = {}

    standard_error = regression_results_stats["std_err"]
    t_value = stats.t.ppf(CI, deg_freedom)
    for ano in lista_anos_de_analise:
        prediction = prediction_data[ano]
        real = variable_values[ano]
        difference = real - prediction
        diff_percent = difference / real

        margin_of_error = t_value * standard_error
        lower_bound = difference - margin_of_error
        upper_bound = difference + margin_of_error

        significant = not (lower_bound <= 0 <= upper_bound)
        p_value_IC = None
        if lower_bound * upper_bound > 0:
            # Não inclui zero no intervalo de confiança
            p_value_IC = 2 * (
                1 - stats.t.cdf(abs(difference / standard_error), deg_freedom)
            )
        else:
            # Inclui zero no intervalo de confiança
            p_value_IC = stats.t.cdf(abs(difference / standard_error), deg_freedom)

        CI_data_dict[ano] = {
            "real_values": real,
            "prediction_values": prediction,
            "difference": difference,
            "difference_percentual": diff_percent,
            "margin_of_error": margin_of_error,
            "lower_bound": lower_bound,
            "upper_bound": upper_bound,
            "significant": significant,
            "p_value_IC": p_value_IC,
        }
        if len(lista_anos_de_analise) == 1:
            return CI_data_dict[ano]

    return CI_data_dict

NameError: name 'PeriodosExcluirAnalise_original' is not defined

In [ ]:
testa = ["1", "2", "3"]
len(testa)
testa_serie = pd.DataFrame(testa)
testa_serie.shape[1]

In [ ]:
df_dados = ExportedDFs["RealDataSeries"]
previsoes = ExportedDFs["PredictionSeries"]
df_P = ExportedDFs["PredictionSeries"]
ResultsStats_DF = ExportedDFs["LinearRegressionResultStats"]
# CI_DataDF = CalcSigCI_RealvsPredicted(df_P, ResultsStats_DF, df_dados)
anos_excluir = []  # PeriodosExcluirAnalise

toplot_prediction_data = {}
# for index, row in CI_DataDF.iterrows():
# toplot_prediction_data[index] = {2020: row.to_dict()}

std_errors = ResultsStats_DF["std_err"].to_dict()
slopes_d = ResultsStats_DF["slope"].to_dict()
p_values_d = ResultsStats_DF["p_value"].to_dict()
r_list = ResultsStats_DF["r_value"].to_dict()
r_sqr_list = ResultsStats_DF["r_squared"].to_dict()
r_sqr_adjst_list = ResultsStats_DF["r_squared_adj"].to_dict()
slope_normalized = ResultsStats_DF["slope_normalized"].to_dict()

tipos_transtorno = df_dados.index.tolist()

# anos_adicionais = [2022,2023,2024,2025,2026]
anos = df_dados.columns
anos_adicionais = []  # anos[-1] + (len(anos) *2)


anos_adicionais_prev = previsoes.shape[1]  # len(anos)-len(PeriodosExcluirAnalise)
# for i in range(anos[-1],anos_adicionais_prev+1):
for i in range(anos_adicionais_prev - len(anos), anos_adicionais_prev + 1):
    anos_adicionais.append(i)

anos_adicionais = df_dados.columns
######################################################################

escala_constante = False
escala_semi_ajustada = False
sort = True


dados_reais = pd.DataFrame(columns=df_P.columns)

dados_reais[df_dados.columns] = df_dados

previsoes = previsoes.reindex(columns=dados_reais.columns, fill_value=np.nan)


ps = dados_reais.index[0]  # dados_reais.iloc[0, 0]  # Primeira linha, primeira coluna
us = dados_reais.index[-1]  # Última linha, primeira coluna


# ps_str = ps.decode('utf-8')  # Convertendo bytes para string usando o encoding 'u

tipo_dado = "INDETERMINADO"
contem_total = us == "Grand Total" or us == "Total"
if ps == "Feminino":
    tipo_dado = "Sexo"
elif (
    ps
    == "(F20 - F29) Esquizofrenia, transtornos esquizotípicos e transtornos delirantes"
):
    tipo_dado = "Transtornos_Agrupamento"
elif us == "80 anos ou mais":
    tipo_dado = "Faixa_Etaria"
elif us == "60 anos ou mais":
    tipo_dado = "Faixa_Etaria_Consolidada"
elif re.match(r"^F\d{2,}$", ps):
    tipo_dado = "CID_individual"


if tipo_dado == "Sexo":
    escala_constante = True
    escala_semi_ajustada = False
    sort = False

if tipo_dado == "Faixa_Etaria":
    sort = False

if tipo_dado == "Faixa_Etaria_Consolidada":
    sort = False

fonte = {"fontname": "DejaVu Serif"}
fonte_legenda = FontProperties(family=fonte["fontname"])

dados_reais_valores = dados_reais.values  # Valores sem cabeçalhos

# Extrair dados de previsões
previsoes_valores = previsoes.values  # Valores sem cabeçalhos

if contem_total:
    dados_reais_valores_calculo = dados_reais_valores[:-1]
    previsoes_valores_calculo = previsoes_valores[:-1]
else:
    dados_reais_valores_calculo = dados_reais_valores
    previsoes_valores_calculo = previsoes_valores

min_value = np.min(
    np.concatenate(
        (
            dados_reais_valores_calculo[: -(anos_adicionais_prev - 1)],
            previsoes_valores_calculo,
        )
    )
)
max_value = (
    np.max(np.concatenate((dados_reais_valores_calculo, previsoes_valores_calculo)))
    * 1.1
)

# Definir a magnitude desejada (diminuída em cerca de 5000 unidades)
magnitude = max_value - min_value
magnitude_desejada = magnitude - 8000

# Ajustar o valor mínimo e máximo para ter a magnitude desejada
min_y_total = min_value  # - (magnitude - magnitude_desejada) / 2
max_y_total = max_value  # + (magnitude - magnitude_desejada) / 2

total_tipos = len(tipos_transtorno)


# Definir o número de colunas e linhas
num_colunas = 2 if total_tipos == 4 else 3
num_linhas = max(2, (total_tipos + num_colunas - 1) // num_colunas)

fig, axs = plt.subplots(
    num_linhas, num_colunas, figsize=(5 * num_colunas, 5 * num_linhas)
)
plt.subplots_adjust(hspace=1)


max_values_dados_reais = dados_reais.max(axis=1)
max_values_previsoes = previsoes.max(axis=1)

if sort:
    # Combine the maximum values and indices into a single DataFrame
    combined_max_df = pd.DataFrame(
        {
            "dados_reais_max": dados_reais.max(axis=1),
            "previsoes_max": previsoes.max(axis=1),
        }
    )

    # Sort the combined DataFrame by the maximum values column
    combined_max_df_sorted = combined_max_df.sort_values(
        by="dados_reais_max", ascending=True
    )

    # Use the sorted indices to sort both original DataFrames
    dados_reais = dados_reais.loc[combined_max_df_sorted.index]
    previsoes = previsoes.loc[combined_max_df_sorted.index]

    dados_reais_valores = dados_reais.values  # Valores sem cabeçalhos
    previsoes_valores = previsoes.values  # Valores sem cabeçalhos

cabecalhos = dados_reais.columns.tolist()  # Ignorar o cabeçalho da primeira coluna
tipos_transtorno = (
    dados_reais.index.tolist()
)  # Índices do DataFrame são os tipos de transtorno


def quebrar_titulo(titulo):
    palavras = titulo.split()
    linhas = []
    linha_atual = ""
    for palavra in palavras:
        if (
            len(linha_atual) + len(palavra) <= 30
        ):  # Adiciona a palavra à linha atual se não exceder o limite
            if linha_atual:
                linha_atual += " "
            linha_atual += palavra
        else:  # Se exceder o limite, inicia uma nova linha
            linhas.append(linha_atual)
            linha_atual = palavra
    if linha_atual:
        linhas.append(linha_atual)
    return "\n".join(linhas)


def check_intersection(bbox1, bbox2):
    x1, y1, width1, height1 = bbox1.bounds
    x2, y2, width2, height2 = bbox2.bounds

    if (
        x1 < x2 + width2
        and x1 + width1 > x2
        and y1 < y2 + height2
        and y1 + height1 > y2
    ):
        return True  # As caixas se intersectam
    else:
        return False  # As caixas não se intersectam


def check_intersection_p(bbox, x_point, y_point):
    # Verificar se a caixa de anotação intersecta o ponto específico
    if bbox.x0 < x_point < bbox.x1 and bbox.y0 < y_point < bbox.y1:
        return True
    else:
        return False


def annotate(ax, x_anno, y_mid, offset):
    ax.annotate(
        f"Significativo\nDiff: {diff_rounded}\nDiff %: {diff_percent_rounded}",
        xy=(x_anno, y_mid),
        xycoords="data",
        xytext=offset,
        textcoords="offset points",
        arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0"),
        fontsize=8.5,
        bbox=dict(facecolor="lightblue", alpha=0.4),
    )


if False:  # total_tipos == 2:
    tipo1, tipo2 = tipos_transtorno

    # Plotar gráfico com duas séries juntas
    figsize = (5, 3) if tipo_dado == "Faixa_Etaria_Consolidada" else (10, 6)
    plt.figure(figsize=figsize)
    plt.plot(
        cabecalhos, dados_reais_valores[0], label=tipo1 + " (Dados Reais)", marker="o"
    )
    plt.errorbar(
        cabecalhos,
        previsoes_valores[0],
        yerr=std_errors[tipo1],
        label=tipo1 + " (Previsões)",
        linestyle="--",
        capsize=10,
    )
    # plt.plot(cabecalhos, previsoes_valores[0], label=tipo1 + ' (Previsões)', marker='o', linestyle= "--")
    plt.plot(
        cabecalhos,
        dados_reais_valores[1],
        label=tipo2 + " (Dados Reais)",
        marker="x",
        **fonte,
    )
    # plt.plot(cabecalhos, previsoes_valores[1], label=tipo2 + ' (Previsões)', marker='x', linestyle= "--")
    plt.errorbar(
        cabecalhos,
        previsoes_valores[1],
        yerr=std_errors[tipo2],
        label=tipo2 + " (Previsões)",
        linestyle="--",
        capsize=10,
    )
    h_ali = 0.01
    for i, (tipo, dados_reais, previsoes, p_val) in enumerate(
        zip(tipos_transtorno, dados_reais_valores, previsoes_valores, p_values_d)
    ):
        p_val = p_values_d[tipo]
        # Verifique se o p-value é menor que 0.05
        if p_val < 0.05:
            # Se sim, defina o peso da fonte como "bold"
            fontweight = "bold"
            fontstyle = "normal"
        elif p_val < 0.1:
            fontweight = "normal"
            fontstyle = "italic"
        else:
            # Se não, mantenha o peso da fonte normal
            fontweight = "normal"
            fontstyle = "normal"

        # Crie as strings para os textos
        text_labels = [
            "Regressão Linear",
            tipo,
            "Erro Padrão (±): {:.2f}".format(std_errors[tipo]),
            "Coef. Angular (β₁): {:.2f} ".format(slopes_d[tipo]),
            "p-Value: {:.4f}".format(p_val),
            "Coef. Correlação (r): {:.3f}".format(r_list[tipo]),
            # "Coef. Determinação (r²): {:.3f}".format(r_sqr_list[tipo])
        ]

        if r_sqr_adjst_list[tipo]:
            text_labels.append(
                "Coef. Determinação Ajustado (r²): {:.3f}".format(
                    r_sqr_adjst_list[tipo]
                )
            )
        else:
            text_labels.append(
                "Coef. Determinação (r²): {:.3f}".format(r_sqr_list[tipo])
            )
        # Adicione os textos ao gráfico
        v_ali = -0.12

        for i, text_label in enumerate(text_labels):
            plt.text(
                h_ali,
                v_ali - i * 0.05,
                text_label,
                fontsize=8,
                fontweight=i == 3 and fontweight or "normal",
                **fonte,
                fontstyle=i == 3 and fontstyle or "normal",
                horizontalalignment="left",
                verticalalignment="center",
                transform=plt.gca().transAxes,
            )
        h_ali = h_ali + 0.3

    plt.xlabel("Ano")
    plt.ylabel("Valores")
    plt.title(tipo_dado)
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{tipo_dado}_graficos_agrupados.pdf", format="pdf")
    plt.savefig(f"{tipo_dado}_graficos_agrupados.svg", format="svg")
    plt.show()
else:
    # Criar gráficos para cada tipo de transtorno
    for i, tipo in enumerate(dados_reais.index):

        serie_dados_reais = dados_reais.loc[tipo].values
        serie_previsoes = previsoes.loc[tipo].values

        linha = i // num_colunas
        coluna = i % num_colunas

        min_serie = np.min(np.minimum(serie_dados_reais, serie_previsoes))
        max_serie = np.max(np.maximum(serie_dados_reais, serie_previsoes))

        # Verificar se a série atual tem um valor máximo menor que 500
        if escala_semi_ajustada:
            if max_serie < 100:
                min_y = min(min_serie - (min_serie * 0.05), 0)
                max_y = 110
            elif max_serie < 300:
                min_y = min(min_serie - (min_serie * 0.05), 100)
                max_y = 330
            elif max_serie < 500:
                min_y = min(min_serie - (min_serie * 0.05), 100)
                max_y = 550
            elif max_serie < 2000:
                min_y = min(min_serie - (min_serie * 0.05), 500)
                max_y = 2200
            elif max_serie < 3000:
                min_y = min(min_serie - (min_serie * 0.05), 500)
                max_y = 3300
            elif max_serie < 6200:
                min_y = min(min_serie - (min_serie * 0.05), 1000)
                max_y = 6200 * 1.1
            elif max_serie < 10000:
                min_y = min(min_serie - (min_serie * 0.05), 1000)
                max_y = 11000
            elif max_serie < 16000:
                min_y = min(min_serie - (min_serie * 0.05), 1000)
                max_y = 16000 * 1.1
            elif max_serie < 22000:
                min_y = min(min_serie - (min_serie * 0.05), 1000)
                max_y = 22000 * 1.1
            elif max_serie < 30000:
                min_y = min(min_serie - (min_serie * 0.05), 1000)
                max_y = 30000 * 1.1
            elif max_serie < 55000:
                min_y = (
                    10000
                    if tipo_dado != "Faixa_Etaria"
                    else min(min_serie - (min_serie * 0.05), 10000)
                )
                max_y = 56000
            else:
                min_y = (
                    min_serie * 0.9
                )  # min(np.minimum(serie_dados_reais, serie_previsoes)) *0.9
                max_y = max_serie * 1.1
        # elif tipo_dado == "Sexo":
        # min_y = 34000
        # max_y - 81000
        elif escala_constante:

            if escala_semi_ajustada and max_serie < 10000:
                escala_info = "_escala_semi_ajustada_LOW"
                # Se sim, ajustar os limites dos eixos y para a série atual
                min_y = min(min_serie, -500)  # min_y_total
                max_y = min(
                    max(max_serie * 1.1, 3000), 6500
                )  ##max_y_total*0.33   # Ajustando para ser 10% maior que o valor máximo
            elif escala_semi_ajustada and max_serie < 1000:
                escala_info = "_escala_semi_ajustada_Extremo_LOW"
                min_y = min(min_serie, -100)
                max_y = 1000
            elif tipo == "Total" or tipo == "Grand Total" and contem_total:
                if escala_semi_ajustada:
                    escala_info = "_escala_semi_ajustada_TOTAL"
                    min_y = (
                        min_serie * 0.95
                    )  # min(np.minimum(serie_dados_reais, serie_previsoes)) *0.9
                    max_y = (
                        max_serie * 1.05
                    )  # max(np.maximum(serie_dados_reais, serie_previsoes)) *1.1
                else:
                    escala_info = "_escala_ajustada_TOTAL"
                    min_y = np.min(
                        np.concatenate((dados_reais_valores, previsoes_valores))
                    )
                    max_y = (
                        np.max(np.concatenate((dados_reais_valores, previsoes_valores)))
                        * 1.1
                    )
            else:
                if escala_semi_ajustada:
                    escala_info = "_escala_semi_ajustada_LOW"
                    min_y = min(min_serie * 0.9, 14000)  # min_y_total
                    max_y = min(max_serie * 1.1, max_y_total)
                else:
                    escala_info = "_escala_constante"
                    # Caso contrário, aplicar os limites calculados para o intervalo total

                    min_y = min_y_total
                    max_y = max_y_total
        else:
            escala_info = "escala_variavel"
            min_y = min(np.minimum(serie_dados_reais, serie_previsoes))
            max_y = max(np.maximum(serie_dados_reais, serie_previsoes)) * 1.1

        titulo_quebrado = quebrar_titulo(tipo)

        axs[linha, coluna].plot(
            cabecalhos,
            serie_dados_reais,
            label="Dados Reais",
            marker="o",
            color="black",
            markersize=4,
        )

        prev_label = "Previsões" if anos_adicionais_prev > 0 else "Dados Esperados"
        color_prev = "green" if anos_adicionais_prev > 0 else "orange"
        axs[linha, coluna].errorbar(
            cabecalhos,
            serie_previsoes,
            yerr=std_errors[tipo],
            label=prev_label,
            color=color_prev,
            linestyle="--",
            capsize=5,
        )
        upper_bound = [previsao + std_errors[tipo] for previsao in serie_previsoes]
        lower_bound = [previsao - std_errors[tipo] for previsao in serie_previsoes]

        color_pand = "blue"
        anos = [int(ano) for ano in cabecalhos]

        indices_anos = [
            cabecalhos.index(ano) for ano in anos_excluir if ano in cabecalhos
        ]

        # Use os índices para plotar uma única linha para todos os anos

        axs[linha, coluna].errorbar(
            [cabecalhos[indice] for indice in indices_anos],
            [serie_previsoes[indice] for indice in indices_anos],
            yerr=[std_errors[tipo] for indice in indices_anos],
            label="Dados Esperados (Pandemia)",
            marker="o",
            color=color_pand,
            linestyle="",
            capsize=5,
            markersize=6,
        )
        """
      axs[linha, coluna].plot([cabecalhos[indice] for indice in indices_anos],
                                  [serie_previsoes[indice] for indice in indices_anos],
                                  label="Dados Esperados (Pandemia)",
                                  marker='o',
                                  color=color_pand,
                                  linestyle="",
                                  markersize = 6)

      """

        p_val = p_values_d[tipo]

        # Verifique se o p-value é menor que 0.05
        if p_val < 0.05:
            # Se sim, defina o peso da fonte como "bold"
            fontweight = "bold"
            fontstyle = "normal"
        elif p_val < 0.1:
            fontweight = "normal"
            fontstyle = "italic"
        else:
            # Se não, mantenha o peso da fonte normal
            fontweight = "normal"
            fontstyle = "normal"

        # Crie as strings para os textos
        l_std_error = "Erro Padrão (±): {:.2f}".format(
            std_errors[tipo]
        )  # Formata o erro padrão com duas casas decimais
        l_slope = "Coef. Angular (β₁): {:.2f}".format(
            slopes_d[tipo]
        )  # Formata o coeficiente angular com duas casas decimais
        l_slope_normal = "Tx. Cresc. Rel. à Média: {:.2f}%".format(
            slope_normalized[tipo] * 100
        )
        l_PV = "p-Value: {:.5f}".format(
            p_val
        )  # Formata o p-value com quatro casas decimais
        l_r = "Coef. Correlação (r): {:.3f}".format(r_list[tipo])
        # l_r_sqr = "Coef. Determinação (r²): {:.3f}".format(r_sqr_list[tipo])

        if r_sqr_adjst_list[tipo]:
            l_r_sqr = "Coef. Determinação Ajustado (r²): {:.3f}".format(
                r_sqr_adjst_list[tipo]
            )
        else:
            l_r_sqr = "Coef. Determinação (r²): {:.3f}".format(r_sqr_list[tipo])
        # Adicione os textos ao gráfico

        # Adicione os textos ao subplot com o peso da fonte correspondente
        axs[linha, coluna].text(
            0.02,
            -0.12,
            "Regressão Linear",
            fontsize=9,
            fontweight="normal",
            horizontalalignment="left",
            verticalalignment="center",
            transform=axs[linha, coluna].transAxes,
            **fonte,
        )
        axs[linha, coluna].text(
            -0.1,
            -0.17,
            l_std_error,
            fontsize=9,
            fontweight="normal",
            horizontalalignment="left",
            verticalalignment="center",
            transform=axs[linha, coluna].transAxes,
            **fonte,
        )
        axs[linha, coluna].text(
            -0.1,
            -0.22,
            l_slope,
            fontsize=9,
            fontweight="normal",
            horizontalalignment="left",
            verticalalignment="center",
            transform=axs[linha, coluna].transAxes,
            **fonte,
        )
        axs[linha, coluna].text(
            -0.1,
            -0.27,
            l_slope_normal,
            fontsize=9,
            fontweight="normal",
            horizontalalignment="left",
            verticalalignment="center",
            transform=axs[linha, coluna].transAxes,
            **fonte,
        )

        # axs[linha, coluna].text(0.00, -0.27, l_PV, fontsize=9, fontweight=fontweight, fontstyle = fontstyle, horizontalalignment='left', verticalalignment='center', transform=axs[linha, coluna].transAxes, **fonte)
        # axs[linha, coluna].text(0.00, -0.32, l_r, fontsize=9,  fontweight='normal', horizontalalignment='left', verticalalignment='center', transform=axs[linha, coluna].transAxes, **fonte)
        # axs[linha, coluna].text(0.00, -0.37, l_r_sqr, fontsize=9,  fontweight='normal', horizontalalignment='left', verticalalignment='center', transform=axs[linha, coluna].transAxes, **fonte)

        axs[linha, coluna].text(
            0.55,
            -0.17,
            l_PV,
            fontsize=9,
            fontweight=fontweight,
            fontstyle=fontstyle,
            horizontalalignment="left",
            verticalalignment="center",
            transform=axs[linha, coluna].transAxes,
            **fonte,
        )
        axs[linha, coluna].text(
            0.55,
            -0.22,
            l_r,
            fontsize=9,
            fontweight="normal",
            horizontalalignment="left",
            verticalalignment="center",
            transform=axs[linha, coluna].transAxes,
            **fonte,
        )
        axs[linha, coluna].text(
            0.55,
            -0.27,
            l_r_sqr,
            fontsize=9,
            fontweight="normal",
            horizontalalignment="left",
            verticalalignment="center",
            transform=axs[linha, coluna].transAxes,
            **fonte,
        )
        # axs[linha, coluna].set_aspect('auto')

        if len(anos_excluir) == 1:  # Caso apenas um ano seja especificado
            ano_especifico = anos_excluir[0]
            if (
                ano_especifico in cabecalhos
            ):  # Verifica se o ano está presente nos cabeçalhos
                indice_ano = cabecalhos.index(ano_especifico)
                axs[linha, coluna].axvspan(
                    cabecalhos[indice_ano] - 0.5,
                    cabecalhos[indice_ano] + 0.5,
                    ymin=0,
                    ymax=1,
                    color="lightblue",
                    alpha=0.3,
                    zorder=-1,
                )

        # axs[linha, coluna].set_yscale("log")
        axs[linha, coluna].set_xlabel("Ano", **fonte)
        """
      if anos_adicionais_prev > 0:
        x_labels = axs[linha, coluna].get_xticklabels()
        #print(x_labels)
        x_labels_int = [int(float(label.get_text())) for label in x_labels]
        #print(x_labels)
        axs[linha, coluna].set_xticklabels(x_labels_int)
      """
        axs[linha, coluna].set_ylabel("Nº de Atendimentos", **fonte)
        axs[linha, coluna].set_title(titulo_quebrado, fontsize=10, **fonte)
        axs[linha, coluna].legend(prop=fonte_legenda)
        axs[linha, coluna].grid(True)

        legenda_pos = plt.gca().get_legend()

        analise = None  # toplot_prediction_data[tipo]

        try:
            for i, (ano_analise, info) in enumerate(analise.items()):
                indice_ano = cabecalhos.index(ano_analise)

                # Calcula a altura total da escala do eixo y
                y_scale_size = max_y - min_y
                y_anterior = serie_dados_reais[indice_ano - 1]
                y_ultima_prev = serie_previsoes[len(serie_previsoes) - 1]

                # Use o índice do ano para encontrar a posição correspondente na série de previsões
                x_anno = cabecalhos[
                    indice_ano
                ]  # A coordenada x para a anotação é o ano analisado
                y_anno = serie_dados_reais[
                    indice_ano
                ]  # A coordenada y para a anotação é o valor da previsão para o ano analisado
                y_prev = serie_previsoes[indice_ano]
                y_mid = (y_anno + y_prev) / 2

                posicao_y_anotacao = (y_mid - min_y) / (y_scale_size)
                posicao_y_anterior = (y_anterior - min_y) / (y_scale_size)
                posicao_y_ultima_prev = (y_ultima_prev - min_y) / (y_scale_size)

                posicao_threshold = 0.6
                axs[linha, coluna].annotate(
                    "",
                    xy=(x_anno, y_prev),
                    xytext=(x_anno, y_anno),
                    arrowprops=dict(
                        arrowstyle="<|-|>", color="black", alpha=0.5, mutation_scale=15
                    ),
                )

                # print(tipo, legenda_pos, posicao_y_anotacao, slopes_d[tipo])
                if posicao_y_anotacao > 0.4:
                    if slopes_d[tipo] > 0:
                        x_offset = 60
                        y_offset = -60
                    else:
                        x_offset = 60
                        y_offset = 60 if legenda_pos == "upper" else -80
                else:
                    if posicao_y_ultima_prev < 0.4:
                        x_offset = -100 if slopes_d[tipo] > 0 else 40
                        y_offset = 40
                    elif slopes_d[tipo] > 0:
                        x_offset = 60
                        y_offset = -30
                    else:
                        x_offset = 60
                        y_offset = 50

                # Arredonde os valores de diferença e diferença percentual
                diff_rounded = round(
                    info["difference"], 2
                )  # Arredonda para 2 casas decimais
                diff_percent_rounded = round(
                    info["difference_percentual"] * 100, 2
                )  # Arredonda para 2 casas decimais
                p_value_anal = "p-Value: {:.5f}".format(info["p_value_IC"])

                # Adicione a anotação ao subplot

                offset = (x_offset, y_offset)
                text_sign = (
                    "IC Significativo\n"
                    if info["significant"]
                    else "IC Não Significativo\n"
                )
                an_text = f"{text_sign}{p_value_anal}\nΔ: {diff_rounded} ({diff_percent_rounded} %)"

                face_color = "lightblue" if info["significant"] else "gray"
                font_weight = "normal"  #'bold' if info["sgnf"] else 'normal'
                fontsize = 8.5 if info["significant"] else 8.5
                annotation = axs[linha, coluna].annotate(
                    an_text,
                    xy=(x_anno, y_mid),
                    xycoords="data",
                    xytext=offset,
                    textcoords="offset pixels",
                    arrowprops=dict(arrowstyle="->", connectionstyle="arc3,rad=0"),
                    fontsize=fontsize,
                    bbox=dict(facecolor=face_color, alpha=0.4),
                    fontweight=font_weight,
                    **fonte,
                )
                bbox_current = annotation.get_window_extent(renderer=None)
        except:
            print("sorrrey no can do the thing")

        # Definir os limites dos eixos y para ter a magnitude desejada

        axs[linha, coluna].set_ylim(min_y, max_y)
        axs[linha, coluna].set_xlim(cabecalhos[0], cabecalhos[len(cabecalhos) - 1])
        # Salvar o gráfico em PDF
        # plt.savefig(f"{escala_info}_{tipo}_gráfico.pdf", format='pdf')
        # plt.show()
        # Salvar o gráfico em formato editável (SVG)
        # plt.savefig(f"{escala_info}_{tipo}_gráfico.svg", format='svg')

    plt.tight_layout()

    plt.subplots_adjust(hspace=0.7)

    dpi = 1000 if len(tipos_transtorno) < 20 else 1

    if anos_excluir == [] and anos_adicionais_prev > 0:
        plt.savefig(
            f"previsoes_futuras_anos_completos_{tipo_dado}_graficos_agrupados.pdf",
            format="pdf",
            dpi=dpi,
        )
        plt.savefig(
            f"previsoes_futuras_anos_completos_{tipo_dado}_graficos_agrupados.svg",
            format="svg",
            dpi=dpi,
        )
    else:
        plt.savefig(f"{tipo_dado}_graficos_agrupados.pdf", format="pdf", dpi=dpi)
        plt.savefig(f"{tipo_dado}_graficos_agrupados.svg", format="svg", dpi=dpi)

    plt.show()
    print("tipo de dado: ", tipo_dado)

In [ ]:
if False:
    for variable, results in results.items():
        print(f"Regression results for variable: {variable}")
        print(results.summary())
        print("\n")  # Add a separator between variables